## Dynamics on Dipolar Pake Pattern

## Goals
Simulate how dynamics change static pake pattern 
## system setup
we use diffusion on the cone as the model system
the bond vector is retreated as H-N bond

## How-to
1. simulte diffusion on a cone to get the bond vector (check)
2. calculate according chemical shift values for each orientation of the bone vector
3. generate Ix or Ix + iIj trajactory: Ix = cos(theta), theta = dt*frequency (rad/s)    frequency = ppm* B0 * 2pi
4. FT to the frequency conponent 
5. figure out how to emsemble average: can we still use Wigner-D matrix 


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import sph_harm
from scipy.spatial.transform import Rotation as R
import sympy as sp
from sympy.physics.quantum.spin import Rotation as WignerRotation
import numpy as np
import math
from math import factorial as fact

# --- Step 1: Simulate vector trajectory on a cone ---
def simulate_vector_on_cone(S2=0.85, tau_c=0.01, dt=1e-4, num_steps=10000):
    cos_theta = np.sqrt((2 * S2 + 1) / 3)
    theta = np.arccos(cos_theta)
    gamma = 1 / tau_c
    sigma = np.sqrt(2 * gamma)
    phi = 0.0
    vectors = np.zeros((num_steps, 3))

    for i in range(num_steps):
        dphi = -gamma * phi * dt + sigma * np.sqrt(dt) * np.random.randn()
        phi += dphi
        x = np.sin(theta) * np.cos(phi)
        y = np.sin(theta) * np.sin(phi)
        z = np.cos(theta)
        vectors[i] = np.array([x, y, z])
    return vectors

In [1]:
#---step 2. Convert H-N orientaion to chemical shift frequency----

r = 1.0e-10  # distance between the two spins in meters 1 å
gamma1 = 2.675e8  # gyromagnetic ratio for proton in /s/T
# gyromagnetic ratio for nitrogen in rad/s/T
gamma2 = 2.713e7  # gyromagnetic ratio for nitrogen-15 in /s/T
gamma3= 6.728e7 # gyro for carbon-13 /s/T

def dipolar_prefactor_Hz(r, gamma1, gamma2):
    mu_0 = 4 * sp.pi * 1e-7  # T·m/A
    hbar = 1.0545718e-34     # J·s
    h = 6.62607015e-34       # Planck's constant in J·s

    # Energy in joules
    H_dip_J = - (mu_0 / (4 * sp.pi)) * (gamma1 * gamma2 * hbar**2) / r**3

    # Convert to Hz: E / h
    H_dip_Hz = H_dip_J / h

    return H_dip_Hz

def euler_from_z_to_vec(v, seq='zyz', degrees=False):
    """
    Calculate Euler angles to rotate (0,0,1) onto unit vector v.
    
    Parameters
    ----------
    v : array-like, shape (3,)
        Target unit vector.
    seq : str
        Euler angle sequence, e.g. 'xyz', 'zyx', 'zxz', etc.
    degrees : bool
        Return angles in degrees if True, radians otherwise.
        
    Returns
    -------
    euler_angles : ndarray, shape (3,)
        Euler angles that rotate (0,0,1) onto v.
    """
    v = np.array(v, dtype=float)
    v /= np.linalg.norm(v)  # ensure unit vector
    
    # initial vector
    z_axis = np.array([0,0,1])
    
    # angle between them
    cos_theta = np.clip(np.dot(z_axis, v), -1.0, 1.0)
    theta = np.arccos(cos_theta)
    
    # if vectors are parallel or anti-parallel
    if np.isclose(theta, 0):
        rot = R.from_rotvec([0,0,0])
    elif np.isclose(theta, np.pi):
        # any perpendicular axis works if opposite
        axis = np.array([1,0,0]) if not np.allclose(z_axis, [1,0,0]) else np.array([0,1,0])
        rot = R.from_rotvec(axis * np.pi)
    else:
        # general case
        axis = np.cross(z_axis, v)
        axis /= np.linalg.norm(axis)
        rot = R.from_rotvec(axis * theta)
    
    # convert to Euler
    euler_angles = rot.as_euler(seq, degrees=degrees)
    return euler_angles

def xyz2freq(vectors, B0):
    '''
    covert xyz unit vector to orientation to calculate dipolar coupling frequency for according orientation
    param:
    vectors: array of shape (n, 3) representing n xyz unit vectors
    B0: magnetic field strength in Telsa

    '''
    # Calculate the euler angles between the vectors and the z-axis

    alpha_vals, beta_vals, gamma_vals = euler_from_z_to_vec(vectors).T


    alpha, beta, gamma = sp.symbols('alpha beta gamma')

    D= dipolar_prefactor_Hz(r, gamma1, gamma2) 

    expr = D*T_rot[0]*np.sqrt(2/3)/2 # use your actual expression index here


    expr_func = sp.lambdify((alpha, beta, gamma), expr, modules="numpy")
    frequencies = np.zeros(vectors.shape[0])

    # Convert angles to frequencies
    for i in range(vectors.shape[0]):
        frequencies[i] = expr_func(alpha_vals[i], beta_vals[i], gamma_vals[i])

    return frequencies


In [ ]:
## Step 3: Calculate Ix trajatory

def calculate_Ix_trajectory(frequencies, time_points):
    """
    Calculate the Ix trajectory based on the dipolar coupling frequencies and time points.

    Parameters
    ----------
    frequencies : array-like, shape (n,)
        Dipolar coupling frequencies.
    time_points : array-like, shape (m,)
        Time points at which to calculate the Ix trajectory.

    Returns
    -------
    Ix_trajectory : ndarray, shape (m,)
        Calculated Ix trajectory.
    """
    Ix_trajectory = np.zeros(time_points.shape)
    for i, t in enumerate(time_points):
        Ix_trajectory[i] = np.sum(frequencies * np.cos(2 * np.pi * frequencies * t))
    return Ix_trajectory
